<a href="https://colab.research.google.com/github/h-aldarmaki/tutorials/blob/main/LDA_PCA_LSA_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This tutorial was prepared for lecture 7 in **AI701: Foundations of Atificial Initellignece** at [MBZUAI](https://mbzuai.ac.ae). 

email: hanan.aldarmaki@mbzuai.ac.ae

# Linear Discriminant Analysis

This is a simple example that demonstrates the steps involved in LDA calculations. 

## 1. The Data 

We will create a toy dataset that consists of 2D points and 2 classes (each class has 5 samples). To be consist with the format used in sklean, we will create one data matrix, X, and a vector of labels, Y. We will use the labels 1 and 2 for the two classes. 

In [ ]:
import numpy as np

#data array
X=np.array([[4,1],[2,4],[2,3], [3,6], [4,4],[9,10],[6,8],[9,5], [8,7], [10,8] ])

#labels vector
Y=np.array([1,1,1,1,1,2,2,2,2,2])


print(X.shape)
print(Y.shape)

The following block defines a function that we will use to plot the data. The input is X, Y, and a title for the plot. 

In [ ]:
#Plotting function
from matplotlib import pyplot as plt
label_ids = {1: 'class1', 2: 'class2'}
def plot_data(X, Y, title):
    ax = plt.subplot(111)
    for label,marker,color in zip(
        range(1,3),('s', 'o'),('blue', 'red')):

        plt.scatter(x=X[Y == label, 0],y=X[Y == label, 1],
                    marker=marker,
                    color=color,
                    alpha=0.5,
                    label=label_ids[label])

    leg = plt.legend(loc='upper right', fancybox=True)
    leg.get_frame().set_alpha(0.5)
    plt.title(title)

    plt.grid()
    plt.tight_layout


In [ ]:
plot_data(X,Y, "Original dataset")
plt.show()

##2. LDA calculations

We now calculate the mean of each class:

$ \mu_j = \frac{1}{n_j} \sum_{x_i \in c_j} x_i$

and the total mean:

$ \mu = \frac{1}{N} \sum_{j=1}^C n_j\mu_j$






In [ ]:
u1=np.mean(X[Y==1], axis=0)
u2=np.mean(X[Y==2], axis=0)

#total mean:
u=(u1+u2)/2

print(u)


### Within Class Variance:

$S_{Wj} = \sum_{x \in c_j}(x-\mu_j)(x-\mu_j)^T$

$S_W = \sum_{j=1}^C S_{Wj}$

In [ ]:
#mean-centered data:
x1_c=X[Y==1]-u1
x2_c=X[Y==2]-u2

sw_1=np.outer(x1_c[0],x1_c[0].T)
sw_2=np.outer(x2_c[0],x2_c[0].T)

for i in range(1,5):
 sw_1+= np.outer(x1_c[i],x1_c[i])
 sw_2+= np.outer(x2_c[i],x2_c[i])

print("Sw using for loop")
print(sw_1)
#alternatively, we can calculate it in one go by matrix multiplication:
print("\nUsing matrix multiplication:")
print(x1_c.T.dot(x1_c))

In [ ]:
Sw=sw_1+sw_2
print(Sw)

### Between Class Variance:

$S_{Bj} = (\mu_j-\mu)(\mu_j-\mu)^T$

$S_B = \sum_{j=1}^C n_j S_{Bj}$

In [ ]:

s1_b=np.outer((u1-u).T, u1-u )
s2_b=np.outer((u2-u).T, u2-u )

Sb=5*s1_b+5*s2_b #the scaling makes no difference here (it only scales the eigenvalues)
print(Sb)

### Find projection W

Calculate the eignevalues/eigenvectors of $S_W^{-1}S_B$


In [ ]:

R=np.linalg.inv(Sw).dot(Sb)
evals,V = np.linalg.eig(R)

for i in range(len(evals)):
    eigvec_sc = V[:,i].reshape(2,1)   
    print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
    print('Eigenvalue {:}: {:.2e}'.format(i+1, evals[i].real))



We will keep only 1 eigenvector, which has the largest eigenvalue. This will be our projection W, which we will use to project the points into a line. 

In [ ]:
#transform
X_lda= X.dot(np.array( [0.91955932 , 0.39295122 ]).T)

plot_data(X,Y, "Original dataset")
plt.plot([0, 0.91955932*11], [0 , 0.39295122*11 ] )
plt.show()

#expand to 2D for plotting, the y values are all set to 0
X_lda_plot = np.vstack([X_lda, np.zeros(X_lda.shape[0])]).T
plot_data(X_lda_plot, Y, "LDA projection")
plt.show()

## Using LDA function from sklearn

The following block shows the steps for using sklearn's implementation of LDA. Note that the details of the implementation in sklearn are slightly different, so the results will appear different, but the plots will be the same (relative positions). 

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()
clf.fit(X, Y)

#transform the data
X_lda_sk = clf.transform(X)
X_lda_sk = np.vstack([X_lda_sk[:,0], np.zeros(X_lda.shape[0])]).T

plot_data(X_lda_sk, Y, title='Default LDA via scikit-learn')
plt.show()

#Principal Component Analysis

We will now implement PCA using the same dataset. PCA is unsupervised, so we will not use the class lables, Y, except for plotting. 

First we will calculate the total mean of the data and subtract it from all data points to calculate the covariance

$\mu = \frac{1}{n}\sum_{i=1}^Nx_i$

$S=\frac{1}{N}\sum_{i=1}^N(x_i-\mu)(x_i-\mu)^T$

In [ ]:
u=np.mean(X, axis=0)
X_c=X-u
X=X.T #transpose the matrix to get 2x10 matrix (as in slides)
S=X.dot(X.T)

Now we find the eigenvalues/eigenvectos of S, and sort them. We will use the first principal component for projection

In [ ]:


evals,V = np.linalg.eig(S)
print(V)
for i in range(len(evals)):
    eigvec_sc = V[:,i].reshape(2,1)   
    print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
    print('Eigenvalue {:}: {:.2e}'.format(i+1, evals[i].real))

p_var=evals[0]/(evals[0]+evals[1])
print("\n percentage of Variance explained by first eigenvector:", p_var*100,"%")

In [ ]:
#transform
W=np.array( [0.72171413 , 0.69219124 ])
X_pca= W.T.dot(X)

plot_data(X.T,Y, "Original dataset")
plt.plot([0, 0.72171413*11], [0 , 0.69219124*11 ] )
plt.show()

#expand to 2D for plotting, the y values are all set to 0
X_pca_plot = np.vstack([X_pca.T, np.zeros(X_pca.T.shape[0])]).T
plot_data(X_pca_plot, Y, "LDA projection")
plt.show()

## Reconstructing the data

In the previous example, we used only the first principal component to reduce the dimensionality of the data. We can reconstruct the original data perfectly if we keep both components, as shown below. If we discard the second component, we can still reconstruct X, but with  loss. Since we drop the second PC, the variance in that direction will be lost completely. 

In [ ]:
W_full = np.array( [[0.72171413 , 0.69219124 ], [-0.69219124, 0.72171413]]).T
X_pca_full = W_full.T.dot(X)
print(X_pca_full.shape, X.shape)
X_reconst = W_full.dot(X_pca_full)
print("Original X:")
print(X)

print("\n reconstructed X from full pca dimensiolns")
print(X_reconst)


print("\n reconstructed X from first PC")
X_reconst = (np.expand_dims(W, axis=1).dot(np.expand_dims(X_pca, axis=1).T))
print(X_reconst)



In [ ]:
plot_data(X.T,Y, "Original/new dataset")
#X_reconst=X_reconst.T
for label,marker,color in zip(
        range(1,3),('s', 'o'),('green', 'black')):

        plt.scatter(x=X_reconst.T[Y == label, 0],y=X_reconst.T[Y == label, 1],
                    marker=marker,
                    color=color,
                    alpha=0.5,
                    label=label_ids[label])
plt.show()

# Latent Semantic Analysis

Here we construct the example shown in the sldies. We have a term-document matrix. We have 9 documents, and 6 words. We show the cosine similarity before and after applying SVD for dimensionality reduction. 


In [ ]:
TD=np.array([[0,1,1,1,0,1,0,0,1],
             [1,0,0,1,1,1,1,1,1],
             [1,0,0,0,0,0,1,0,0],
             [0,1,0,1,0,0,1,0,1],
             [0,0,1,0,1,1,0,0,0],
             [0,0,0,0,0,1,0,1,0]])

print("Before SVD:")

cosine=TD[:,0].dot(TD[:, 1])/(norm(TD[:,0])*norm(TD[:,1]))
print("cosine similarity of first two documents :", cosine)

cosine=TD[2,:].dot(TD[3,:])/(norm(TD[2,:])*norm(TD[3,:]))
print("cosine similarity of words at index 2 and 3 (funny, hilarious) :", cosine)

cosine=TD[2,:].dot(TD[4,:])/(norm(TD[2,:])*norm(TD[4,:]))
print("cosine similarity of words at index 2 and 4 (funny, sad) :", cosine)

In [ ]:
U,L,V=np.linalg.svd(TD)
from numpy.linalg import norm
print("After SVD and reducing dimensionality to 2:")

w1=V[0:2,0]
w2=V[0:2,1]
cosine=w1.dot(w2)/(norm(w1)*norm(w2))
print("cosine similarity between first two documents:", cosine)

w1=U[2,0:2]
w2=U[3,0:2]
cosine=w1.dot(w2)/(norm(w1)*norm(w2))
print("cosine similarity between the words (funny, hilarious):", cosine)

w1=U[2,0:2]
w2=U[4,0:2]
cosine=w1.dot(w2)/(norm(w1)*norm(w2))
print("cosine similarity between the words (funny, sad):", cosine)


